In [438]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [439]:
sns.set(font="AppleGothic", 
        rc={"axes.unicode_minus":False},
        style='darkgrid')

In [440]:
os.getcwd()

'/Users/chance/Desktop/LH_home/ksc'

In [687]:
folder = os.listdir()

file = []
for f in folder:
    if "_2022" in f:
        data = pd.read_csv("./" + f, encoding="cp949", skiprows=15)
        data = data[list(data.columns[:11])]
        file.append(data)

gajwa = pd.concat(file)
# gajwa = gajwa.dropna(axis=0)

geo = pd.read_csv("./geocoding.csv")
gajwa["경도"] = geo.X
gajwa["위도"] = geo.Y

gajwa = gajwa.fillna({'건축년도':gajwa['건축년도'].mode()[0]})
gajwa = gajwa.fillna({'도로명':gajwa['도로명'].mode()[0]})

# 도로조건을 정수로 변환
road=[]
for a in gajwa["도로조건"]:
    if a=="-":
        a = "8"
    road.append(int(a.split("m")[0]))
gajwa["도로조건"] = road


# 년월 column 생성
year=[]
month=[]
for d in gajwa["계약년월"]:
    day = pd.to_datetime(d, format="%Y%m")
    year.append(day.year)
    month.append(day.month)

gajwa["year"] = year
gajwa["month"] = month

built_day = [pd.to_datetime(d, format="%Y").year for d in gajwa["건축년도"]]
gajwa["건축년도"] = built_day

# 위치 column만들기
gajwa["location"] = gajwa.시군구 + " " + gajwa.도로명


#원핫인코딩으로 전월세 구분하기
gajwa = pd.get_dummies(gajwa, columns = ["전월세구분"])
# gajwa = pd.get_dummies(gajwa, columns = ["도로명"])

# 계약년월 기준으로 전월세전환율 입력하기
rate = pd.read_csv("./rate.csv", encoding="cp949")
rate = rate.T
rate = rate[3:]

month = []
for r in rate.index:
    new = "".join(r.split("."))
    month.append(int(new))

rate["계약년월"] = month
rate.columns = ["전월세전환율", "계약년월"]
rate.head()

gajwa = pd.merge(left=gajwa, right=rate, how="inner", on="계약년월")

keep = [int("".join(k.split(","))) for k in gajwa["보증금(만원)"]]
gajwa["보증금(만원)"] = keep


# 전월세전환율 기준으로 보증금+월세의 가치를 전세로 치환하여 value column으로 만들어준다.
value=[round(gajwa["월세(만원)"][i] * 12 / (gajwa["전월세전환율"][i]/100) + gajwa["보증금(만원)"][i]) for i in range(len(gajwa))]
gajwa["전세전환가격"] = value
#gajwa["도로별 전세값평균"] = gajwa.groupby("도로명").전세전환가격.transform("mean")


# 도로명 결측치 제거
indexNames = []
for i in range(len(gajwa["도로명"])):
    if "번길" not in gajwa["도로명"][i]:
        indexNames.append(i)

gajwa.drop(indexNames, inplace=True)

gajwa

,시군구,번지,도로조건,계약면적(㎡),계약년월,계약일,보증금(만원),월세(만원),건축년도,도로명,경도,위도,year,month,location,전월세구분_월세,전월세구분_전세,전월세전환율,전세전환가격
0,경상남도 진주시 가좌동,1***,8,12.00,202003,5,1500,25,2007,진주대로404번길,128.105889,35.157466,2020,3,경상남도 진주시 가좌동 진주대로404번길,1,0,4.8,7750
1,경상남도 진주시 가좌동,1***,8,15.00,202003,11,500,40,2016,가좌길78번길,128.105857,35.157800,2020,3,경상남도 진주시 가좌동 가좌길78번길,1,0,4.8,10500
5,경상남도 진주시 가좌동,5**,12,19.80,202003,14,200,36,2011,내동로348번길,128.106121,35.158214,2020,3,경상남도 진주시 가좌동 내동로348번길,1,0,4.8,9200
6,경상남도 진주시 가좌동,1***,8,20.00,202003,26,300,30,2016,가좌길48번길,128.105857,35.157800,2020,3,경상남도 진주시 가좌동 가좌길48번길,1,0,4.8,7800
7,경상남도 진주시 가좌동,1***,12,20.10,202003,8,300,30,2014,진주대로404번길,128.105966,35.156961,2020,3,경상남도 진주시 가좌동 진주대로404번길,1,0,4.8,7800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3166,경상남도 진주시 가좌동,1***,8,25.00,202011,28,5000,0,2017,진주대로404번길,128.105857,35.157800,2020,11,경상남도 진주시 가좌동 진주대로404번길,0,1,4.7,5000
3167,경상남도 진주시 가좌동,5**,12,30.00,202011,24,300,37,2007,진주대로500번길,128.108498,35.154904,2020,11,경상남도 진주시 가좌동 진주대로500번길,1,0,4.7,9747
3168,경상남도 진주시 가좌동,1***,8,30.00,202011,27,2000,20,2019,내동로320번길,128.107180,35.162495,2020,11,경상남도 진주시 가좌동 내동로320번길,1,0,4.7,7106
3169,경상남도 진주시 가좌동,1***,12,36.30,202011,26,300,55,1996,가좌길36번길,128.109160,35.157148,2020,11,경상남도 진주시 가좌동 가좌길36번길,1,0,4.7,14343


In [688]:
# # 전세 월세 구분 데이터셋
# gajwa = gajwa[gajwa.전월세구분 == "월세"]
# gajwa = pd.DataFrame.drop(gajwa, columns=["전월세구분"], axis=1)
# gajwa

## 지도

In [764]:
import math as m

def distance(x1, x2, y1, y2):
    radius = 6371 # 지구 반지름(km)
    toRadian = m.pi / 180

    deltaLatitude = abs(x1 - x2) * toRadian
    deltaLongitude = abs(y1 - y2) * toRadian

    sinDeltaLat = m.sin(deltaLatitude / 2)
    sinDeltaLng = m.sin(deltaLongitude / 2)
    squareRoot = m.sqrt(sinDeltaLat * sinDeltaLat + m.cos(x1 * toRadian) * m.cos(x2 * toRadian) * sinDeltaLng * sinDeltaLng)

    distance = 2 * radius * m.asin(squareRoot)

    return distance

In [776]:
gnu_long = 35.1530444
gnu_lat = 128.1010899

dist = []
for long, lat in zip(gajwa["경도"], gajwa["위도"]):
    d = distance(lat, long, gnu_lat, gnu_long)
    dist.append(d)

np.min(dist)

6822.227895108833

In [748]:
import folium

m = folium.Map(location=[gnu_long, gnu_lat], zoom_start=20)
folium.Marker([gnu_long, gnu_lat]).add_to(m)

In [749]:
gajwa_data = pd.DataFrame.drop(gajwa, columns=["시군구","계약년월", "번지", "도로명", "계약일", "보증금(만원)", "월세(만원)", "location", "경도", "위도"], axis=1)

gajwa_data

,도로조건,계약면적(㎡),건축년도,year,month,전월세구분_월세,전월세구분_전세,전월세전환율,전세전환가격,distance
0,8,12.00,2007,2020,3,1,0,4.8,7750,6824.774167
1,8,15.00,2016,2020,3,1,0,4.8,10500,6824.746434
5,12,19.80,2011,2020,3,1,0,4.8,9200,6824.724392
6,8,20.00,2016,2020,3,1,0,4.8,7800,6824.746434
7,12,20.10,2014,2020,3,1,0,4.8,7800,6824.817255
...,...,...,...,...,...,...,...,...,...,...
3166,8,25.00,2017,2020,11,0,1,4.7,5000,6824.746434
3167,12,30.00,2007,2020,11,1,0,4.7,9747,6825.083558
3168,8,30.00,2019,2020,11,1,0,4.7,7106,6824.428700
3169,12,36.30,1996,2020,11,1,0,4.7,14343,6824.932898


# 머신러닝

In [750]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from xgboost import plot_importance
import lightgbm as lgb
import catboost
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score

In [751]:
X = gajwa_data.drop(["전세전환가격"], axis=1)
y = gajwa_data['전세전환가격']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=1645)

scaler = StandardScaler()
X_train_standardized = scaler.fit_transform(X_train)
X_test_standardized = scaler.fit_transform(X_test)

minmaxScaler = MinMaxScaler()
X_train_standardized2 = minmaxScaler.fit_transform(X_train)
X_test_standardized2 = minmaxScaler.fit_transform(X_test)

## 선형회귀분석

In [752]:
# lr = LinearRegression()
# lr.fit(X_train, y_train)
# y_preds = lr.predict(X_test)

# mse = mean_squared_error(y_test, y_preds)
# rmse = np.sqrt(mse)

# print(f"MSE : {mse:.3f}", f"RMSE: {rmse:.3f}")
# print(f"variance score : {r2_score(y_test, y_preds):.3f}")

In [753]:
# lr.fit(X_train_standardized, y_train)
# lr.score(X_test_standardized, y_test)

## Ridge

In [754]:
# R_model = Ridge()
# R_model.fit(X_train, y_train)
# print(R_model.score(X_test, y_test))

# R_model.fit(X_train_standardized, y_train)
# R_model.score(X_test_standardized, y_test)

## Lasso

In [755]:
# L_model = Lasso(alpha=0.5)
# L_model.fit(X_train, y_train)
# print(L_model.score(X_test, y_test))

# L_model.fit(X_train_standardized, y_train)
# L_model.score(X_test_standardized, y_test)

## ElasticNet

In [756]:
# ela = ElasticNet(alpha=0.5)
# ela.fit(X_train, y_train)
# print(ela.score(X_test, y_test))

# ela.fit(X_train_standardized, y_train)
# ela.score(X_test_standardized, y_test)

## 랜덤포레스트

In [757]:
randomforest = RandomForestRegressor(random_state=0, n_jobs=-1)
randomforest_model = randomforest.fit(X_train, y_train)
print(randomforest_model.score(X_test, y_test))

randomforest_model = randomforest.fit(X_train_standardized, y_train)
print(randomforest_model.score(X_test_standardized, y_test))

0.6834876980438676
0.6914673744724977


In [758]:
gbm = GradientBoostingRegressor()

gbm.fit(X_train, y_train)
print(gbm.score(X_test, y_test))
gbm.fit(X_train_standardized, y_train)
print(gbm.score(X_test_standardized, y_test))

0.6886053276843125
0.6966661216150114


In [759]:
gajwa_copy = pd.DataFrame.drop(gajwa, columns=["시군구","계약년월", "번지", "도로명", "계약일", "보증금(만원)", "월세(만원)", "location", "전월세전환율"], axis=1)
gajwa_copy

X = gajwa_copy.drop(["전세전환가격"], axis=1)
y = gajwa_copy['전세전환가격']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=1645)

scaler = StandardScaler()
X_train_standardized = scaler.fit_transform(X_train)
X_test_standardized = scaler.fit_transform(X_test)

## LightGBM

In [760]:
lightgbm = lgb.LGBMRegressor()

lightgbm.fit(X_train, y_train)
print(lightgbm.score(X_test, y_test))

lightgbm.fit(X_train_standardized, y_train)
print(lightgbm.score(X_test_standardized, y_test))

0.6867141247235289
0.6846829287670985
0.2601675567268258


## XGBoost

In [730]:
xgboost = XGBRegressor(n_estimators=50)
xgboost.fit(X_train, y_train)
print(xgboost.score(X_test, y_test))

xgboost.fit(X_train_standardized, y_train)
xgboost.score(X_test_standardized, y_test)

0.6174481282814579


/Users/chance/opt/anaconda3/envs/py3/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.64624452533467

## CatBoost

In [731]:
cb = catboost.CatBoostRegressor()
cb.fit(X_train_standardized, y_train)
cb.score(X_test_standardized, y_test)

Learning rate set to 0.046193
0:	learn: 2813.4214882	total: 5.62ms	remaining: 5.62s
1:	learn: 2749.9306177	total: 6.11ms	remaining: 3.05s
2:	learn: 2691.4791697	total: 6.55ms	remaining: 2.18s
3:	learn: 2637.8375990	total: 6.96ms	remaining: 1.73s
4:	learn: 2582.5760609	total: 7.44ms	remaining: 1.48s
5:	learn: 2530.0817789	total: 7.87ms	remaining: 1.3s
6:	learn: 2484.7403209	total: 8.28ms	remaining: 1.17s
7:	learn: 2439.0792307	total: 8.68ms	remaining: 1.08s
8:	learn: 2397.4304313	total: 8.97ms	remaining: 988ms
9:	learn: 2352.4286611	total: 9.37ms	remaining: 928ms
10:	learn: 2313.7126553	total: 9.97ms	remaining: 897ms
11:	learn: 2279.5303830	total: 10.4ms	remaining: 856ms
12:	learn: 2243.9153240	total: 10.8ms	remaining: 822ms
13:	learn: 2210.1687630	total: 11.3ms	remaining: 795ms
14:	learn: 2183.5618856	total: 11.7ms	remaining: 768ms
15:	learn: 2152.8910130	total: 12.1ms	remaining: 746ms
16:	learn: 2126.7278574	total: 12.6ms	remaining: 731ms
17:	learn: 2100.7756149	total: 13.1ms	remainin

0.6881680766018674

## 데이터시각화

In [575]:
corr = gajwa_data.corr()
corr

,도로조건,계약면적(㎡),건축년도,year,month,전월세구분_월세,전월세구분_전세,전세전환가격
도로조건,1.000000,0.003002,-0.104286,0.245077,0.042160,0.047291,-0.047291,0.176671
계약면적(㎡),0.003002,1.000000,-0.093352,-0.202356,0.089829,-0.012119,0.012119,0.392502
건축년도,-0.104286,-0.093352,1.000000,0.142690,-0.028468,-0.101014,0.101014,-0.001156
year,0.245077,-0.202356,0.142690,1.000000,-0.032791,-0.026964,0.026964,0.240234
month,0.042160,0.089829,-0.028468,-0.032791,1.000000,-0.040222,0.040222,0.041322
전월세구분_월세,0.047291,-0.012119,-0.101014,-0.026964,-0.040222,1.000000,-1.000000,0.476071
전월세구분_전세,-0.047291,0.012119,0.101014,0.026964,0.040222,-1.000000,1.000000,-0.476071
전세전환가격,0.176671,0.392502,-0.001156,0.240234,0.041322,0.476071,-0.476071,1.000000


## 카카오맵api